In [0]:
import gc
# Clear specific large variables from memory
for var in ['model', 'pipeline', 'rf', 'predictions', 'train_df', 'test_df']:
    if var in locals():
        del locals()[var]

gc.collect()
print("Python memory cleared. Proceed to the next cell.")

Python memory cleared. Proceed to the next cell.


In [0]:
# 1. Imports
import mlflow
import mlflow.spark
from pyspark.ml import Pipeline
from pyspark.ml.feature import StringIndexer, VectorAssembler, OneHotEncoder
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.sql.functions import col
from pyspark.ml.functions import vector_to_array

In [0]:
# 2. Load and Cast Data
# Loading the Gold features table prepared in the previous notebook
gold_df = spark.read.table("healthcare.gold.readmission_features")

# List of numeric columns to be cast to Double for the VectorAssembler
numeric_cols = [
    "time_in_hospital", "num_lab_procedures", "num_procedures", 
    "num_medications", "number_diagnoses", "number_outpatient", 
    "number_emergency", "number_inpatient"
]

for c in numeric_cols:
    gold_df = gold_df.withColumn(c, col(c).cast("double"))

In [0]:
# 3. Explicitly Define String Indexers
# These stages convert string categories into numerical indices with your requested suffix (_idx)
age_indexer = StringIndexer(inputCol="age", outputCol="age_idx", handleInvalid="keep")
gender_indexer = StringIndexer(inputCol="gender", outputCol="gender_idx", handleInvalid="keep")
race_indexer = StringIndexer(inputCol="race", outputCol="race_idx", handleInvalid="keep")

# Start building the pipeline stages list
stages = [age_indexer, gender_indexer, race_indexer]

# 4. Define Feature Columns List
# This is your specific list used for the VectorAssembler
feature_cols = [
    "age_idx",
    "gender_idx",
    "race_idx",
    "time_in_hospital",
    "num_lab_procedures",
    "num_procedures",
    "num_medications",
    "number_outpatient",
    "number_emergency",
    "number_inpatient",
    "number_diagnoses"
]

In [0]:
# 5. Build Vector Assembler
# Combines all features into a single 'features' vector for the model
assembler = VectorAssembler(inputCols=feature_cols, outputCol="features")
stages += [assembler]

In [0]:
# 6. Define the UC Volume path for staging (Required for Serverless)
uc_volume_path = "/Volumes/healthcare/bronze/raw_volume/ml_tmp"

In [0]:
# 7. MLflow Tracking & Model Training
experiment_path = f"/Users/{spark.sql('SELECT current_user()').collect()[0][0]}/Readmission_Prediction"
mlflow.set_experiment(experiment_path)

with mlflow.start_run(run_name="RF_Final_Integrated"):
    
    # Lightweight Classifier for Serverless (10 trees, depth 5)
    rf = RandomForestClassifier(labelCol="label", featuresCol="features", numTrees=10, maxDepth=5)
    stages += [rf]
    
    # Create the Pipeline and Split Data
    pipeline = Pipeline(stages=stages)
    train_df, test_df = gold_df.randomSplit([0.8, 0.2], seed=42)
    
    # Train the Model
    model = pipeline.fit(train_df)
    
    # Transform test data to get predictions
    predictions = model.transform(test_df)
    
    # Extract Readmission Risk ---
    # probability[1] is the likelihood of the patient being readmitted (label 1)
    predictions = predictions.withColumn(
        "readmission_risk", 
        vector_to_array(col("probability"))[1]
    )
    
    # Evaluate Performance (AUC)
    evaluator = BinaryClassificationEvaluator(labelCol="label")
    auc = evaluator.evaluate(predictions)
    
    # Log metrics and model artifacts to MLflow
    mlflow.log_metric("auc", auc)
    mlflow.spark.log_model(
        spark_model=model, 
        artifact_path="readmission_rf_model",
        dfs_tmpdir=uc_volume_path
    )
    
    print(f"SUCCESS! Model Training Complete. AUC Score: {auc}")

2026/01/30 17:32:55 WARNING mlflow.utils.requirements_utils: Found pyspark version (4.0.0+databricks.connect.17.2.2) contains a local version label (+databricks.connect.17.2.2). MLflow logged a pip requirement for this package as 'pyspark==4.0.0' without the local version label to make it installable from PyPI. To specify pip requirements containing local version labels, please use `conda_env` or `pip_requirements`.
2026/01/30 17:33:00 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: /local_disk0/user_tmp_data/spark-73426648-1824-48cc-9f79-11/tmp0b47eidm/model, flavor: spark). Fall back to return ['pyspark==4.0.0']. Set logging level to DEBUG to see the full traceback. 
2026/01/30 17:33:00 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


SUCCESS! Model Training Complete. AUC Score: 0.6186876848159526


In [0]:
# 8. FINAL WRITE: Save predictions with exactly 5 columns
(
    predictions.select(
        "patient_nbr", 
        "label", 
        "prediction", 
        "probability", 
        "readmission_risk"
    )
    .write
    .format("delta")
    .mode("overwrite")
    .option("overwriteSchema", "true") 
    .saveAsTable("healthcare.gold.readmission_predictions")
)

In [0]:
%sql
DESCRIBE TABLE healthcare.gold.readmission_predictions;

col_name,data_type,comment
patient_nbr,string,null
label,int,null
prediction,double,null
probability,vector,null
readmission_risk,double,null
